In [1]:
import os
import datetime
from zoneinfo import ZoneInfo
import pandas as pd
import numpy as np
import ffmpeg
import PIL
from PIL import Image
from matplotlib import cm

In [2]:
camera_path = os.path.join('data', 'camera_data') # Infra Red camera data'
mat_path = os.path.join('data', 'mat_data') # ..\Pressure Mat Data'

# centre_camera_path = os.path.join(camera_path, 'tests', 'subject1', 'Centre 1')
# centre_mat_path = os.path.join(mat_path, 'tests', 'subject1', 'Centre 1')

In [3]:
centre_camera_path = os.path.join(camera_path, 'tests', 'subject1', 'Centre 1')
centre_mat_path = os.path.join(mat_path, 'tests', 'subject1', 'Centre 1')

mat_csv_file = 'Centre.csv'

outfile_name = 'centre1'

camera_frame_number = 2
mat_frame_number = 700

In [4]:
centre_camera_path = os.path.join(camera_path, 'tests', 'subject1', 'Corner 1')
centre_mat_path = os.path.join(mat_path, 'tests', 'subject1', 'Corner 1')

mat_csv_file = 'Corner 1.csv'

outfile_name = 'corner1'

camera_frame_number = 65
mat_frame_number = 540

In [5]:
centre_camera_path = os.path.join(camera_path, 'tests', 'subject1', 'Corner 2')
centre_mat_path = os.path.join(mat_path, 'tests', 'subject1', 'Corner 2')

mat_csv_file = 'Corner 2.csv'

outfile_name = 'corner2'

camera_frame_number = 65
mat_frame_number = 580

In [6]:
centre_camera_path = os.path.join(camera_path, 'tests', 'subject1', 'Corner 3')
centre_mat_path = os.path.join(mat_path, 'tests', 'subject1', 'Corner 3')

mat_csv_file = 'Corner 3.csv'

outfile_name = 'corner3'

camera_frame_number = 75
mat_frame_number = 470

In [7]:
centre_camera_path = os.path.join(camera_path, 'tests', 'subject1', 'Corner 4')
centre_mat_path = os.path.join(mat_path, 'tests', 'subject1', 'Corner 4')

mat_csv_file = 'Corner 4.csv'

outfile_name = 'corner4'

camera_frame_number = 90
mat_frame_number = 410

In [8]:
def camera_date_reader(d):
    try:
        return datetime.datetime.strptime(d, 'frame_%Y-%m-%dT%H_%M_%S.%f.csv')
    except ValueError:
        return datetime.datetime.strptime(d, 'frame_%Y-%m-%dT%H_%M_%S.csv')

In [9]:
def mat_date_reader(d):
    try:
        return datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S.%f%z')
    except ValueError:
        return datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S%z')

In [10]:
def vidwrite_camera(fn, images, framerate=8, vcodec='libx264'):
    if not isinstance(images, np.ndarray):
        images = np.asarray(images)
    n,height,width = images.shape
    process = (
        ffmpeg
            .input('pipe:', format='rawvideo', pix_fmt='gray', s=f'{width}x{height}')
            .output(fn, pix_fmt='yuv420p', vcodec=vcodec, r=framerate,
                    vf="drawtext=fontfile=Arial.ttf: text=%{n}: x=0: y=h-(2*lh): fontcolor=white: box=1: boxcolor=0x00000099"
                   )
            .overwrite_output()
            .run_async(pipe_stdin=True)
    )
    for frame in images:
        process.stdin.write(
            (frame * 255)
                .astype(np.uint8)
                .tobytes()
        )
    process.stdin.close()
    process.wait()

In [11]:
def vidwrite_mat(fn, images, framerate=30, vcodec='libx264'):
    if not isinstance(images, np.ndarray):
        images = np.asarray(images)
    n,height,width = images.shape
    width += 1
    process = (
        ffmpeg
            .input('pipe:', format='rawvideo', pix_fmt='rgb24', s=f'{width}x{height}')
            .output(fn, pix_fmt='yuv420p', vcodec=vcodec, r=framerate,
                    vf="drawtext=fontfile=Arial.ttf: text=%{n}: x=0: y=5: fontcolor=white: box=1: boxcolor=0x00000099"
                   )
            .overwrite_output()
            .run_async(pipe_stdin=True)
    )
    for frame in images:
        mod_frame = np.hstack((frame, np.zeros((64, 1), dtype=frame.dtype)))
        process.stdin.write(
            (cm.hot(mod_frame) * 255)[:, : , :-1]
                .astype(np.uint8)
                .tobytes()
        )
    process.stdin.close()
    process.wait()

In [12]:
# def vidwrite(fn, images, framerate=8, vcodec='libx264'):
#     if not isinstance(images, np.ndarray):
#         images = np.asarray(images)
#     n,height,width,channels = images.shape
#     process = (
#         ffmpeg
#             .input('pipe:', format='rawvideo', pix_fmt='rgb24', s=f'{width}x{height}')
#             .output(fn, pix_fmt='yuv420p', vcodec=vcodec, r=framerate)
#             .overwrite_output()
#             .run_async(pipe_stdin=True)
#     )
#     for frame in images:
#         process.stdin.write(
#             frame
#                 .astype(np.uint8)
#                 .tobytes()
#         )
#     process.stdin.close()
#     process.wait()

In [13]:
bst_tz = datetime.timezone(datetime.timedelta(hours=1), name='BST')

In [14]:
test_dt = datetime.datetime.strptime('frame_2024-05-17T09_33_45.713423.csv', 'frame_%Y-%m-%dT%H_%M_%S.%f.csv')
test_dt

datetime.datetime(2024, 5, 17, 9, 33, 45, 713423)

In [15]:
# test_dt.replace(tzinfo='UTC+01:00')

In [16]:
test_dt.replace(tzinfo=ZoneInfo('Europe/London'))

datetime.datetime(2024, 5, 17, 9, 33, 45, 713423, tzinfo=zoneinfo.ZoneInfo(key='Europe/London'))

In [17]:
test_dt.replace(tzinfo=bst_tz)

datetime.datetime(2024, 5, 17, 9, 33, 45, 713423, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600), 'BST'))

In [18]:
camera_files = {}
for f in os.listdir(centre_camera_path):
    fp = os.path.join(centre_camera_path, f)
    if os.path.isfile(fp) and 'ContDrc' not in f:
        tm = camera_date_reader(f).replace(tzinfo=ZoneInfo('Europe/London'))
        camera_files[tm] = fp
        # print(f, tm)
camera_files = {t: camera_files[t] for t in sorted(camera_files)}
camera_files

{datetime.datetime(2024, 5, 17, 9, 46, 12, 129384, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_12.129384.csv',
 datetime.datetime(2024, 5, 17, 9, 46, 12, 303979, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_12.303979.csv',
 datetime.datetime(2024, 5, 17, 9, 46, 12, 392852, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_12.392852.csv',
 datetime.datetime(2024, 5, 17, 9, 46, 12, 548644, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_12.548644.csv',
 datetime.datetime(2024, 5, 17, 9, 46, 12, 737678, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_12.737678.csv',
 datetime.datetime(2024, 5, 17, 9, 46, 12, 850362, tzinfo=zoneinfo.ZoneInfo(key='Europe/London')): '

In [19]:
time_sorted_filenames = [camera_files[t] for t in sorted(camera_files)]
time_sorted_filenames

['data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_12.129384.csv',
 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_12.303979.csv',
 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_12.392852.csv',
 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_12.548644.csv',
 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_12.737678.csv',
 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_12.850362.csv',
 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_13.021597.csv',
 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_13.097318.csv',
 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_13.266757.csv',
 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_13.391926.csv',
 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_13.557368.csv',
 'data/camera_data/tests/subject1/Corner 4/frame_2024-05-17T09_46_13.676098.csv',
 'data/camera_da

In [20]:
tfile = time_sorted_filenames[0]

In [21]:
ctimes = [t for t in camera_files]
diffs = [b - a for a, b in zip(ctimes, ctimes[1:])]
diffs

[datetime.timedelta(microseconds=174595),
 datetime.timedelta(microseconds=88873),
 datetime.timedelta(microseconds=155792),
 datetime.timedelta(microseconds=189034),
 datetime.timedelta(microseconds=112684),
 datetime.timedelta(microseconds=171235),
 datetime.timedelta(microseconds=75721),
 datetime.timedelta(microseconds=169439),
 datetime.timedelta(microseconds=125169),
 datetime.timedelta(microseconds=165442),
 datetime.timedelta(microseconds=118730),
 datetime.timedelta(microseconds=138757),
 datetime.timedelta(microseconds=123829),
 datetime.timedelta(microseconds=170484),
 datetime.timedelta(microseconds=118255),
 datetime.timedelta(microseconds=165456),
 datetime.timedelta(microseconds=120469),
 datetime.timedelta(microseconds=175096),
 datetime.timedelta(microseconds=131269),
 datetime.timedelta(microseconds=218162),
 datetime.timedelta(microseconds=122494),
 datetime.timedelta(microseconds=107550),
 datetime.timedelta(microseconds=202992),
 datetime.timedelta(microseconds=116

In [22]:
len(diffs), len(diffs[5:-5])

(227, 217)

In [23]:
sum(diffs[5:-5], start=datetime.timedelta(0)) / len(diffs[5:-5])

datetime.timedelta(microseconds=141031)

In [24]:
camera_frames = []
for f in time_sorted_filenames:
    arr = np.loadtxt(f, delimiter=',', usecols=list(range(80)), max_rows=60).reshape((1, 60, 80))
    camera_frames.append(arr)
camera_frames = np.vstack(camera_frames)
camera_frames.shape

(228, 60, 80)

In [25]:
cmin, cmax = np.min(camera_frames), np.max(camera_frames)
camera_frames = (camera_frames - cmin) / (cmax - cmin)
camera_frames

array([[[0.09824561, 0.09824561, 0.13333333, ..., 0.15789474,
         0.14035088, 0.17192982],
        [0.09824561, 0.07017544, 0.25964912, ..., 0.22807018,
         0.16491228, 0.08421053],
        [0.06315789, 0.13333333, 0.18596491, ..., 0.16842105,
         0.24210526, 0.05964912],
        ...,
        [0.1754386 , 0.18245614, 0.19649123, ..., 0.21403509,
         0.20350877, 0.13684211],
        [0.18245614, 0.10526316, 0.23508772, ..., 0.16140351,
         0.21052632, 0.08421053],
        [0.1122807 , 0.14736842, 0.18947368, ..., 0.16842105,
         0.20701754, 0.1122807 ]],

       [[0.13333333, 0.13333333, 0.13333333, ..., 0.13333333,
         0.25964912, 0.18947368],
        [0.14035088, 0.09122807, 0.15438596, ..., 0.16491228,
         0.15789474, 0.03859649],
        [0.09824561, 0.09473684, 0.13333333, ..., 0.15438596,
         0.25964912, 0.14385965],
        ...,
        [0.13333333, 0.16491228, 0.19649123, ..., 0.15087719,
         0.2245614 , 0.08421053],
        [0.1

In [46]:
cmin, cmax

(7886.0, 8171.0)

In [26]:
tframe = pd.read_csv(tfile, header=None, skipfooter=1, engine='python').drop(80, axis='columns')
tframe.shape

(60, 80)

In [27]:
tframe.loc[0]

0     7914
1     7914
2     7924
3     7928
4     7926
      ... 
75    7932
76    7945
77    7931
78    7926
79    7935
Name: 0, Length: 80, dtype: int64

In [28]:
mframe = pd.read_csv(os.path.join(centre_mat_path,mat_csv_file), 
                     converters={'Timestamp': mat_date_reader},
                    index_col='Frame')
mframe.dtypes

Timestamp           datetime64[ns, UTC+01:00]
Range Min (mmHg)                        int64
Range Max (mmHg)                        int64
0                                       int64
1                                       int64
                              ...            
1723                                    int64
1724                                    int64
1725                                    int64
1726                                    int64
1727                                    int64
Length: 1731, dtype: object

In [29]:
mframe

,Timestamp,Range Min (mmHg),Range Max (mmHg),0,1,2,3,4,5,6,...,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727
Frame,,,,,,,,,,,,,,,,,,,,,
1,2024-05-17 09:46:08.473000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2024-05-17 09:46:08.505000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2024-05-17 09:46:08.537000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2024-05-17 09:46:08.570000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2024-05-17 09:46:08.602000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,2024-05-17 09:46:56.966000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1498,2024-05-17 09:46:56.999000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1499,2024-05-17 09:46:57.031000+01:00,0,104,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
len(time_sorted_filenames)

228

In [31]:
# mframe.loc[1000]['Timestamp']

In [32]:
# [t.replace(tzinfo=ZoneInfo('Europe/London')) <= mframe.loc[1000]['Timestamp'] for t in camera_files]

In [33]:
mframe['Timestamp']

Frame
1      2024-05-17 09:46:08.473000+01:00
2      2024-05-17 09:46:08.505000+01:00
3      2024-05-17 09:46:08.537000+01:00
4      2024-05-17 09:46:08.570000+01:00
5      2024-05-17 09:46:08.602000+01:00
                     ...               
1497   2024-05-17 09:46:56.966000+01:00
1498   2024-05-17 09:46:56.999000+01:00
1499   2024-05-17 09:46:57.031000+01:00
1500   2024-05-17 09:46:57.064000+01:00
1501   2024-05-17 09:46:57.096000+01:00
Name: Timestamp, Length: 1501, dtype: datetime64[ns, UTC+01:00]

In [34]:
mat_frames = mframe.drop(['Timestamp', 'Range Min (mmHg)', 'Range Max (mmHg)'], axis='columns')
mat_frames = (mat_frames.to_numpy().astype(np.float64) / 104).reshape(-1, 64, 27)
mat_frames

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [35]:
((cm.hot(mat_frames[0]) * 255)[... , :-1]
     .astype(np.uint8)
     # .tobytes()
)

array([[[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]],

       [[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]],

       [[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]],

       ...,

       [[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]],

       [[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]],

       [[10,  0,  0],
        [10,  0,  0],
        [10,  0,  0],
        ...,
        [10,  0,  0],
        [10,  0,  0],
        [10,  0,  0]]], dtype=uint8)

In [36]:
mat_frames.shape

(1501, 64, 27)

In [37]:
for frame in mat_frames:
    print(len((cm.hot(frame) * 255)[... , :-1]
            .astype(np.uint8)
            .tobytes()
    ))

5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184
5184


In [38]:
mat_frames[0].shape

(64, 27)

In [39]:
b = np.hstack((mat_frames[0], np.zeros((64, 1), dtype=mat_frames.dtype)))
b.shape

(64, 28)

In [40]:
vidwrite_camera(f'{outfile_name}.mp4', camera_frames)

ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-7)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libope

In [41]:
vidwrite_mat(f'{outfile_name}_mat.mp4', mat_frames)

ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-7)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716144998863/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libope

In [42]:
camera_frames.dtype

dtype('float64')

In [43]:
mat_frames.dtype

dtype('float64')

In [44]:
Image.fromarray((camera_frames[camera_frame_number] * 255).astype(np.uint8)).save(f'{outfile_name}_camera.png')

In [45]:
Image.fromarray((mat_frames[mat_frame_number] * 255).astype(np.uint8)).save(f'{outfile_name}_mat.png')